In [ ]:
import dotenv
import os
dotenv.load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

In [ ]:
from crewai import LLM

llm=LLM(
    model="gpt-3.5-turbo",
    temperature=0.7,
    max_tokens=200,
)

In [ ]:
from crewai.tools import BaseTool

# BaseTool is a class for all CrewAI custom tools
#Crew AI lets you build tools that agents can use to interact with the world. These tools can be anything from a search engine, a calculator, a database, or even an API. By defining tools, you can enable your agents to perform specific tasks and access information that they wouldn't be able to access on their own.
#every custom tool should inherit from BaseTool .
class ReplaceJargonsTool(BaseTool):
    name: str = "Replace Jargons Tool"
    description: str = "A tool to replace technical jargons with simple language for better understanding."

    def _run(self, email: str) -> str:
        # a dictionary of corporate jargons and their simple language equivalents
        replacements = {
            "PRX": "Press Release",
            "ASAP": "As Soon As Possible",
            "TAS": "Theoretical Approach to Strategy",
            "SDS" : "Strategic Decision Support",
            "EOD": "End of Day",
            "WIP": "Work in Progress",
            "OOO": "Out of Office",
            "ETA": "Estimated Time of Arrival",
            "KPI": "Key Performance Indicator",
            "ROI": "Return on Investment",
            "POC" : "Proof of Concept",
            "ping": "Contact"
        }

        suggestions = [] # create an empty list to store suggestions
        email = """Hi Team,
Please find the PRX for the new product launch attached. We need to finalize the details ASAP to ensure a successful launch. Let's schedule a meeting to discuss the TAS and SDS for the launch strategy. Also, please provide an update on the WIP for the marketing materials. OOO next week, so let's aim to have everything ready by EOD today. ETA for the final version of the PRX is tomorrow. Let's make sure we hit our KPIs and achieve a strong ROI with this launch. If you have any questions or need further clarification, please ping me.
Best,
[Your Name]"""
        email_lower = email.lower() # convert the email to lowercase for case-insensitive matching
        for jargon, replacement in replacements.items():
            if jargon.lower() in email_lower:
                suggestions.append(f"Consider replacing  '{jargon}' with '{replacement}'") # add a suggestion to the list   
            return "\n".join(suggestions) if suggestions else "No jargons found in the email." # join the suggestions into a single string and return it
# create an instance of the ReplaceJargonsTool
replace_jargons_tool = ReplaceJargonsTool()

original_email = """Hi Team,
Please find the PRX for the new product launch attached. We need to finalize the details ASAP to ensure a successful launch. Let's schedule a meeting to discuss the TAS and SDS for the launch strategy. Also, please provide an update on the WIP for the marketing materials. OOO next week, so let's aim to have everything ready by EOD today. ETA for the final version of the PRX is tomorrow. Let's make sure we hit our KPIs and achieve a strong ROI with this launch. If you have any questions or need further clarification, please ping me.
Best,
[Your Name]"""
# suggestions = replace_jargons_tool.run(original_email)
# print(suggestions)



In [ ]:
from crewai import Agent, Task, Crew

# Define Agents
email_assistant = Agent(
    role="Email assistant with jargon replacement capabilities",
    goal="An agent that helps users write clear and jargon-free emails by suggesting replacements for technical terms.",
    llm=llm,
    backstory="You summarize like a consultant and also help replace technical jargons with simple language",
    tools=[replace_jargons_tool],
    verbose=True
)


# Define Tasks
email_task = Task(
    name="Email Task",
    description=f"Take the following email and suggest replacements for any technical jargons found in it:\n\n{original_email}",
    expected_output="a professional email with all jargons replaced by simple language and a list of suggestions for each jargon found in the original email.",
    agent=email_assistant
)



In [ ]:
# Create a Crew and add tasks
crew = Crew(
    name="Email Assistance Crew",
    agents=[email_assistant],
    description="A professional crew that helps users write clear and jargon-free emails.",
    tasks=[email_task]
    )

In [ ]:
#Run
result = crew.kickoff()
print(result)
